# 13 - Data Engineering & ETL Pipelines

Budowanie produkcyjnych pipeline'ów przetwarzania danych.

**Tematy:**
- ETL pipeline: Raw → Clean → Transform → Load
- Data quality checks - walidacja danych
- Delta Lake - ACID, time travel, schema evolution, MERGE
- SCD Type 2 - śledzenie zmian w danych
- Medallion architecture (Bronze → Silver → Gold)
- Zapis do wielu formatów i systemów

## 1. Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

spark = SparkSession.builder \
    .appName("13_Data_Engineering") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", 
            "org.postgresql:postgresql:42.7.1,"
            "io.delta:delta-spark_2.12:3.3.1") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

# Katalogi
BRONZE_PATH = "/tmp/datalake/bronze"
SILVER_PATH = "/tmp/datalake/silver"
GOLD_PATH = "/tmp/datalake/gold"

## 2. Medallion Architecture

Standardowy wzorzec organizacji danych w data lake:

```
Bronze (Raw)     → Silver (Clean)      → Gold (Aggregated)
─────────────      ───────────────        ─────────────────
Surowe dane        Oczyszczone           Gotowe do analiz
Bez zmian          Deduplikacja          Agregaty
Append-only        Type casting          Metryki biznesowe
Schema-on-read     Walidacja             Feature store
```

## 3. Bronze Layer - surowe dane

Załaduj dane as-is, dodaj metadane ingestii.

In [ ]:
# Załaduj surowe dane z PostgreSQL
ratings_raw = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies_raw = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

# Dodaj metadane ingestii
ratings_bronze = ratings_raw \
    .withColumn("_ingestion_timestamp", current_timestamp()) \
    .withColumn("_source", lit("postgresql")) \
    .withColumn("_batch_id", lit("batch_001"))

movies_bronze = movies_raw \
    .withColumn("_ingestion_timestamp", current_timestamp()) \
    .withColumn("_source", lit("postgresql")) \
    .withColumn("_batch_id", lit("batch_001"))

# Zapisz do Bronze (Parquet)
ratings_bronze.write.mode("overwrite").parquet(f"{BRONZE_PATH}/ratings")
movies_bronze.write.mode("overwrite").parquet(f"{BRONZE_PATH}/movies")

print(f"Bronze ratings: {ratings_bronze.count()} rows")
print(f"Bronze movies: {movies_bronze.count()} rows")
ratings_bronze.show(3)

## 4. Data Quality Checks

Przed transformacją do Silver - walidacja jakości danych.

In [ ]:
class DataQualityChecker:
    """Framework do walidacji jakości danych."""
    
    def __init__(self, df, name):
        self.df = df
        self.name = name
        self.results = []
    
    def check_not_null(self, columns):
        """Sprawdź czy kolumny nie mają nulli."""
        for col_name in columns:
            null_count = self.df.filter(col(col_name).isNull()).count()
            total = self.df.count()
            passed = null_count == 0
            self.results.append({
                "check": f"not_null({col_name})",
                "passed": passed,
                "detail": f"{null_count}/{total} nulls ({null_count/total*100:.2f}%)"
            })
        return self
    
    def check_range(self, column, min_val, max_val):
        """Sprawdź czy wartości są w zakresie."""
        out_of_range = self.df.filter(
            (col(column) < min_val) | (col(column) > max_val)
        ).count()
        passed = out_of_range == 0
        self.results.append({
            "check": f"range({column}, {min_val}-{max_val})",
            "passed": passed,
            "detail": f"{out_of_range} out of range"
        })
        return self
    
    def check_unique(self, columns):
        """Sprawdź unikalność kombinacji kolumn."""
        total = self.df.count()
        distinct = self.df.select(columns).distinct().count()
        duplicates = total - distinct
        passed = duplicates == 0
        self.results.append({
            "check": f"unique({','.join(columns)})",
            "passed": passed,
            "detail": f"{duplicates} duplicates"
        })
        return self
    
    def check_referential_integrity(self, column, ref_df, ref_column):
        """Sprawdź integralność referencyjną."""
        orphans = self.df.join(
            ref_df.select(col(ref_column).alias("_ref")),
            col(column) == col("_ref"),
            "left_anti"
        ).count()
        passed = orphans == 0
        self.results.append({
            "check": f"ref_integrity({column})",
            "passed": passed,
            "detail": f"{orphans} orphan records"
        })
        return self
    
    def report(self):
        """Wyświetl raport."""
        print(f"\n{'='*60}")
        print(f"Data Quality Report: {self.name}")
        print(f"{'='*60}")
        all_passed = True
        for r in self.results:
            status = "PASS" if r['passed'] else "FAIL"
            if not r['passed']:
                all_passed = False
            print(f"  [{status}] {r['check']}: {r['detail']}")
        print(f"\nOverall: {'ALL PASSED' if all_passed else 'FAILURES DETECTED'}")
        return all_passed

In [ ]:
# Odczytaj bronze
ratings_b = spark.read.parquet(f"{BRONZE_PATH}/ratings")
movies_b = spark.read.parquet(f"{BRONZE_PATH}/movies")

# Walidacja ratings
ratings_ok = DataQualityChecker(ratings_b, "ratings_bronze") \
    .check_not_null(["user_id", "movie_id", "rating"]) \
    .check_range("rating", 0.5, 5.0) \
    .check_range("user_id", 1, 999999) \
    .check_unique(["user_id", "movie_id"]) \
    .check_referential_integrity("movie_id", movies_b, "movie_id") \
    .report()

# Walidacja movies
movies_ok = DataQualityChecker(movies_b, "movies_bronze") \
    .check_not_null(["movie_id", "title", "genres"]) \
    .check_unique(["movie_id"]) \
    .report()

## 5. Silver Layer - oczyszczone dane

Transformacje: czyszczenie, type casting, deduplication, enrichment.

In [ ]:
# Silver - ratings
ratings_silver = ratings_b \
    .select("user_id", "movie_id", "rating", "rating_timestamp") \
    .dropDuplicates(["user_id", "movie_id"]) \
    .filter(col("rating").between(0.5, 5.0)) \
    .filter(col("user_id").isNotNull() & col("movie_id").isNotNull()) \
    .withColumn("rating_date", to_date(col("rating_timestamp"))) \
    .withColumn("rating_year", year(col("rating_timestamp"))) \
    .withColumn("rating_month", month(col("rating_timestamp"))) \
    .withColumn("rating_hour", hour(col("rating_timestamp"))) \
    .withColumn("is_positive", (col("rating") >= 4.0).cast("int")) \
    .withColumn("_processed_at", current_timestamp())

# Silver - movies (enriched)
movies_silver = movies_b \
    .select("movie_id", "title", "genres") \
    .dropDuplicates(["movie_id"]) \
    .withColumn("year", regexp_extract(col("title"), r"\((\d{4})\)", 1).cast("int")) \
    .withColumn("clean_title", regexp_replace(col("title"), r"\s*\(\d{4}\)\s*$", "")) \
    .withColumn("genre_array", split(col("genres"), "\\|")) \
    .withColumn("num_genres", size(split(col("genres"), "\\|"))) \
    .withColumn("primary_genre", element_at(split(col("genres"), "\\|"), 1)) \
    .withColumn("decade", (floor(col("year") / 10) * 10).cast("int")) \
    .withColumn("_processed_at", current_timestamp())

# Zapisz Silver
ratings_silver.write.mode("overwrite") \
    .partitionBy("rating_year") \
    .parquet(f"{SILVER_PATH}/ratings")

movies_silver.write.mode("overwrite") \
    .parquet(f"{SILVER_PATH}/movies")

print(f"Silver ratings: {ratings_silver.count()} rows")
print(f"Silver movies: {movies_silver.count()} rows")
ratings_silver.show(3)
movies_silver.show(3, truncate=False)

## 6. Gold Layer - agregaty biznesowe

Gotowe do konsumpcji przez analizy, dashboardy, modele ML.

In [ ]:
# Odczytaj Silver
ratings_s = spark.read.parquet(f"{SILVER_PATH}/ratings")
movies_s = spark.read.parquet(f"{SILVER_PATH}/movies")

# Gold: Movie Stats - statystyki per film
gold_movie_stats = ratings_s.groupBy("movie_id").agg(
    count("*").alias("total_ratings"),
    countDistinct("user_id").alias("unique_raters"),
    round(avg("rating"), 3).alias("avg_rating"),
    round(stddev("rating"), 3).alias("std_rating"),
    round(avg("is_positive"), 3).alias("positive_rate"),
    min("rating_date").alias("first_rating_date"),
    max("rating_date").alias("last_rating_date")
).join(movies_s, "movie_id")

# Wilson score for ranking (lower bound of confidence interval)
@udf("double")
def wilson_score(positive, total):
    import math
    if total == 0: return 0.0
    z = 1.96
    p = positive / total
    denom = 1 + z * z / total
    centre = p + z * z / (2 * total)
    delta = z * math.sqrt((p * (1 - p) + z * z / (4 * total)) / total)
    return float((centre - delta) / denom)

gold_movie_stats = gold_movie_stats.withColumn(
    "wilson_score",
    round(wilson_score(
        (col("positive_rate") * col("total_ratings")).cast("double"),
        col("total_ratings").cast("double")
    ), 4)
)

gold_movie_stats.write.mode("overwrite").parquet(f"{GOLD_PATH}/movie_stats")

print("Gold Movie Stats - Top 15 (Wilson Score):")
gold_movie_stats.filter(col("total_ratings") >= 100) \
    .orderBy(desc("wilson_score")) \
    .select("title", "total_ratings", "avg_rating", "positive_rate", "wilson_score") \
    .show(15, truncate=False)

In [ ]:
# Gold: User Profiles - profil każdego użytkownika
gold_user_profiles = ratings_s.groupBy("user_id").agg(
    count("*").alias("total_ratings"),
    round(avg("rating"), 2).alias("avg_rating"),
    round(stddev("rating"), 2).alias("rating_variance"),
    sum("is_positive").alias("positive_count"),
    countDistinct("movie_id").alias("unique_movies"),
    min("rating_date").alias("first_activity"),
    max("rating_date").alias("last_activity"),
    countDistinct("rating_year").alias("active_years")
).withColumn(
    "user_segment",
    when(col("total_ratings") >= 1000, "power_user")
    .when(col("total_ratings") >= 100, "active")
    .when(col("total_ratings") >= 20, "casual")
    .otherwise("new")
).withColumn(
    "positivity_rate",
    round(col("positive_count") / col("total_ratings"), 2)
)

gold_user_profiles.write.mode("overwrite").parquet(f"{GOLD_PATH}/user_profiles")

print("User segment distribution:")
gold_user_profiles.groupBy("user_segment").agg(
    count("*").alias("count"),
    round(avg("avg_rating"), 2).alias("segment_avg_rating"),
    round(avg("total_ratings"), 0).alias("avg_num_ratings")
).orderBy(desc("count")).show()

In [ ]:
# Gold: Genre Trends - trendy gatunków w czasie
genre_exploded = ratings_s.join(movies_s.select("movie_id", "genre_array"), "movie_id") \
    .withColumn("genre", explode(col("genre_array")))

gold_genre_trends = genre_exploded.groupBy("genre", "rating_year").agg(
    count("*").alias("num_ratings"),
    round(avg("rating"), 2).alias("avg_rating"),
    countDistinct("movie_id").alias("unique_movies"),
    countDistinct("user_id").alias("unique_users")
)

gold_genre_trends.write.mode("overwrite") \
    .partitionBy("genre") \
    .parquet(f"{GOLD_PATH}/genre_trends")

print("Genre trends (Drama, last 5 years):")
gold_genre_trends.filter(
    (col("genre") == "Drama") & (col("rating_year") >= 2010)
).orderBy("rating_year").show()

### Zadanie 1
Stwórz Gold table: `gold_recommendations_input` z następującymi kolumnami per user:
- user_id, avg_rating, total_ratings, user_segment
- top_3_genres (3 najczęściej oceniane gatunki)
- avg_movie_year (średni rok filmów, które ocenił)

Ten DataFrame potem mógłby zasilać model rekomendacji.

In [ ]:
# Twoje rozwiązanie:


## 7. Delta Lake

Delta Lake dodaje do data lake:
- **ACID transactions** - atomowe zapisy
- **Schema enforcement** - walidacja schematu
- **Time travel** - dostęp do poprzednich wersji danych
- **MERGE** (upsert) - wstaw lub zaktualizuj
- **Z-ordering** - optymalizacja layoutu plików

In [ ]:
from delta.tables import DeltaTable

DELTA_PATH = "/tmp/datalake/delta"

# Zapisz jako Delta
movies_s = spark.read.parquet(f"{SILVER_PATH}/movies")

movies_s.write \
    .format("delta") \
    .mode("overwrite") \
    .save(f"{DELTA_PATH}/movies")

print("Delta table created")

# Odczytaj Delta
delta_movies = spark.read.format("delta").load(f"{DELTA_PATH}/movies")
delta_movies.show(5, truncate=False)

In [ ]:
# MERGE (upsert) - aktualizuj istniejące, wstaw nowe
# Symulujmy aktualizację: zmiana gatunku kilku filmów + nowy film

updates = spark.createDataFrame([
    # Aktualizacja istniejącego
    (1, "Toy Story (1995)", "Adventure|Animation|Children|Comedy|Fantasy|Family",
     1995, "Toy Story", None, 6, "Adventure", 1990, None),
    # Nowy film
    (999999, "Test Movie (2025)", "Sci-Fi|Action",
     2025, "Test Movie", None, 2, "Sci-Fi", 2020, None)
], movies_s.schema)

delta_table = DeltaTable.forPath(spark, f"{DELTA_PATH}/movies")

delta_table.alias("target") \
    .merge(
        updates.alias("source"),
        "target.movie_id = source.movie_id"
    ) \
    .whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

# Sprawdź
spark.read.format("delta").load(f"{DELTA_PATH}/movies") \
    .filter(col("movie_id").isin(1, 999999)) \
    .select("movie_id", "title", "genres", "num_genres") \
    .show(truncate=False)

In [ ]:
# Time Travel - dostęp do poprzednich wersji

# Historia wersji
delta_table.history().select("version", "timestamp", "operation", "operationMetrics").show(truncate=False)

In [ ]:
# Odczytaj starszą wersję (przed MERGE)
old_version = spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load(f"{DELTA_PATH}/movies")

current_version = spark.read.format("delta").load(f"{DELTA_PATH}/movies")

print(f"Wersja 0 (przed MERGE): {old_version.count()} filmów")
print(f"Wersja aktualna: {current_version.count()} filmów")

# Toy Story w starej wersji
print("\nToy Story - wersja 0 (oryginał):")
old_version.filter(col("movie_id") == 1).select("title", "genres", "num_genres").show(truncate=False)

print("Toy Story - wersja aktualna (po MERGE):")
current_version.filter(col("movie_id") == 1).select("title", "genres", "num_genres").show(truncate=False)

In [ ]:
# Schema Evolution - dodawanie kolumn
# Delta domyślnie blokuje zmiany schematu

enriched_movies = current_version \
    .withColumn("popularity_tier", lit("unknown"))

# Z mergeSchema = true pozwala na dodanie nowej kolumny
enriched_movies.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(f"{DELTA_PATH}/movies")

print("Schema po ewolucji:")
spark.read.format("delta").load(f"{DELTA_PATH}/movies").printSchema()

## 8. SCD Type 2 - Slowly Changing Dimensions

Śledzenie historii zmian rekordu.

Każdy rekord ma:
- `effective_from` - od kiedy ten wiersz jest aktualny
- `effective_to` - do kiedy (null = aktualny)
- `is_current` - czy to aktualna wersja

In [ ]:
from datetime import datetime

# Inicjalizuj tabelę SCD2
movies_scd = movies_s \
    .select("movie_id", "title", "genres", "primary_genre") \
    .withColumn("effective_from", lit(datetime(2020, 1, 1)).cast("timestamp")) \
    .withColumn("effective_to", lit(None).cast("timestamp")) \
    .withColumn("is_current", lit(True))

movies_scd.write.format("delta").mode("overwrite").save(f"{DELTA_PATH}/movies_scd2")

print(f"SCD2 initial: {movies_scd.count()} rows")
movies_scd.filter(col("movie_id") == 1).show(truncate=False)

In [ ]:
# Symuluj aktualizację: Toy Story zmienia gatunek
updates_scd = spark.createDataFrame([
    (1, "Toy Story (1995)", "Adventure|Animation|Children|Comedy|Fantasy|Family", "Adventure"),
    (2, "Jumanji (1995)", "Adventure|Children|Fantasy|Action", "Adventure"),
], ["movie_id", "title", "genres", "primary_genre"])

now = current_timestamp()

# SCD Type 2 MERGE:
# 1. Zamknij stary rekord (is_current = false, effective_to = now)
# 2. Wstaw nowy rekord (is_current = true, effective_from = now)

delta_scd = DeltaTable.forPath(spark, f"{DELTA_PATH}/movies_scd2")

# Krok 1: Zamknij stare rekordy
delta_scd.alias("target") \
    .merge(
        updates_scd.alias("source"),
        "target.movie_id = source.movie_id AND target.is_current = true"
    ) \
    .whenMatchedUpdate(set={
        "is_current": lit(False),
        "effective_to": now
    }) \
    .execute()

# Krok 2: Wstaw nowe wersje
new_records = updates_scd \
    .withColumn("effective_from", now) \
    .withColumn("effective_to", lit(None).cast("timestamp")) \
    .withColumn("is_current", lit(True))

new_records.write.format("delta").mode("append").save(f"{DELTA_PATH}/movies_scd2")

# Sprawdź historię Toy Story
print("Historia Toy Story (SCD Type 2):")
spark.read.format("delta").load(f"{DELTA_PATH}/movies_scd2") \
    .filter(col("movie_id") == 1) \
    .orderBy("effective_from") \
    .show(truncate=False)

## 9. Zapis do wielu systemów

In [ ]:
# Odczytaj Gold stats
movie_stats = spark.read.parquet(f"{GOLD_PATH}/movie_stats")
user_profiles = spark.read.parquet(f"{GOLD_PATH}/user_profiles")

# Zapis do PostgreSQL - tabela analityczna
movie_stats.select(
    "movie_id", "title", "primary_genre", "total_ratings",
    "avg_rating", "wilson_score", "decade"
).write \
    .mode("overwrite") \
    .jdbc(jdbc_url, "analytics.movie_stats", properties=properties)

user_profiles.select(
    "user_id", "total_ratings", "avg_rating", "user_segment",
    "positivity_rate", "first_activity", "last_activity"
).write \
    .mode("overwrite") \
    .jdbc(jdbc_url, "analytics.user_profiles", properties=properties)

print("Zapisano do PostgreSQL: analytics.movie_stats, analytics.user_profiles")

In [ ]:
# Zapis do różnych formatów
movie_stats_small = movie_stats.filter(col("total_ratings") >= 100).limit(1000)

# Parquet (kolumnowy, kompresja, szybki do analiz)
movie_stats_small.write.mode("overwrite").parquet("/tmp/exports/parquet")

# CSV (czytelny, kompatybilny)
movie_stats_small.write.mode("overwrite") \
    .option("header", True) \
    .csv("/tmp/exports/csv")

# JSON (API-friendly)
movie_stats_small.write.mode("overwrite").json("/tmp/exports/json")

print("Eksport zakończony")

## Zadanie końcowe

Zbuduj kompletny **ETL Pipeline** z:

1. **Bronze**: Załaduj dane z PostgreSQL z metadanymi ingestii
2. **Quality Checks**: Walidacja (nulls, ranges, referential integrity)
3. **Silver**: Oczyszczenie, enrichment, partycjonowanie
4. **Gold**: Stwórz 3 tabele:
   - `daily_activity` - dzienna aktywność (oceny, unikalni użytkownicy)
   - `genre_popularity` - ranking gatunków per dekada
   - `user_movie_matrix` - sparse matrix user×genre z średnią oceną (dla ML)
5. **Delta Lake**: Zapisz Gold tables jako Delta z MERGE (upsert)
6. **PostgreSQL**: Wyeksportuj Gold do tabel analitycznych

In [ ]:
# Twoje rozwiązanie:


In [ ]:
# Cleanup
import shutil
shutil.rmtree("/tmp/datalake", ignore_errors=True)
shutil.rmtree("/tmp/exports", ignore_errors=True)

spark.stop()